# Creating a DB/Table

## Databases

```mysql
CREATE DATABASE [IF NOT EXISTS] db_name 
CHARACTER SET charset_name
COLLATE collation_name;
```

After creating a database, check it out:

```mysql
SHOW CREATE DATABASE db_name;
```

## Tables

### CREATE TABLE [IF NOT EXISTS]

```mysql
CREATE TABLE IF NOT EXISTS tbl_name (
    id INT UNSIGNED NOT NULL AUTO_INCREMENT,
    x1 VARCHAR(20),
    x2 ENUM('this','that'),
    x3 DATE,
    x4 TEXT,
    x5 TINYINT NOT NULL DEFAULT 0,
    x6 TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    x7 DATE DEFAULT CURRENT_DATE,
    
    PRIMARY KEY (id),
    INDEX (...),
    FOREIGN KEY (...) REFERENCES parent_tbl_name (...)
) 
ENGINE = INNODB
CHARACTER SET character_set_name
COLLATE collation_name;
```

* AUTO_INCREMENT should be either PRIMARY KEY or UNIQUE index, and it must be NOT NULL.


* AUTO_INCREMENT starts with 1. If we insert a NULL value or 0 into the INT AUTO_INCREMENT column, the value of the column is set to the next sequence value. 


* AUTO_INCREMENT column does not accept negative values.


### CREATE TEMPORARY TABLE

A temporary table can have the same name as a normal table in the database. If tbl is a normal table that already exists in the database and we create a temporary table with the same name tbl, then the name tbl is the name of the temporary table. When it is dropped, the normal table with the same name can be accessed.

We can use DROP TABLE tbl to drop a temporary table, but it will be better to use DROP TEMPORARY TABLE tbl to avoid dropping a normal table.

```mysql
CREATE TEMPORARY TABLE tbl(
    ...
);

CREATE TEMPORARY TABLE tbl LIKE other_tbl;

CREATE TEMPORARY TABLE tbl SELECT ...;
```


### CREATE TABLE ... LIKE

```mysql
CREATE TABLE new_tbl LIKE old_tbl;
```

### CREATE TABLE ... SELECT ...

```mysql
CREATE TABLE new_tbl SELECT * FROM old_tbl;
```

### VALUES ROW(..), ROW(..), ... 

```mysql
SELECT * FROM (VALUES ROW(4,-2),ROW(5,9)) t;
+----------+----------+
| column_0 | column_1 |
+----------+----------+
|        4 |       -2 |
|        5 |        9 |
+----------+----------+
```

## Generated columns

Format: 

col_name data_type [GENERATED ALWAYS] AS (...) [VIRTUAL | STORED] [UNIQUE [KEY]]

* VIRTUAL: not stored physically; default
* STORED: stored physically


```mysql
CREATE TABLE tbl (
    first_name VARCHAR(10) NOT NULL,
    last_name VARCHAR(10) NOT NULL,
    full_name VARCHAR(21) GENERATED ALWAYS AS (CONCAT(first_name,' ',last_name)),
    ...
);
```

# ALTER TABLE

```mysql
ALTER DATABASE db_name
CHARACTER SET character_set_name
COLLATE collation_name;
```

The following clauses follow `ALTER TABLE tbl_name`:

```mysql
ENGINE = INNODB 
CHARACTER SET character_set_name 
COLLATE collation_name;

ADD [COLUMN] d DATE;
ADD x VARCHAR(30), y VARCHAR(50);                # add multiple columns
ADD x DOUBLE GENERATED ALWAYS AS (y*z) STORED;   # add generated column
ADD d VARCHAR(100) NOT NULL FIRST;   # position: as the first column
ADD d TINYINT AFTER c;               # position: after the column c 

ADD INDEX index_name (index_column ASC);
ADD UNIQUE INDEX index_name (index_column);  # unique index
ADD PRIMARY KEY (p), ADD INDEX (i), ADD UNIQUE (u,v);

ADD CONSTRAINT fk_parent_id FOREIGN KEY (parent_id) 
  REFERENCES parent (id);

ADD CONSTRAINT fk_parent_id FOREIGN KEY (parent_id) 
  REFERENCES parent (id)
  ON UPDATE CASCADE
  ON DELETE CASCADE;

DROP [COLUMN] c;
DROP COLUMN c1, DROP COLUMN c2;
DROP INDEX index_name;
DROP FOREIGN KEY fk_symbol;  
# fk_symbol is the name of the foreign key constraint, not the name of the column.
# Find it from SHOW CREATE TABLE tbl_name.

# MODIFY: change data type/properties
MODIFY d TIMESTAMP NOT NULL, MODIFY y VARCHAR(50) AFTER z;
MODIFY x SMALLINT NULL         # remove the NOT NULL constraint from x
MODIFY x VARCHAR(50) CHARACTER SET utf8;

# RENAME: change name
RENAME [TO] t1;                # Change the table name to t1.
RENAME COLUMN c TO d;          # Change simply the column name c to d.

# CHANGE: change name & data type/properties
CHANGE COLUMN curr_name new_name VARCHAR(50) NOT NULL AFTER another_col;
```


# RENAME TABLE

```mysql
RENAME TABLE curr_tbl_name TO new_tbl_name;

RENAME TABLE t1 TO new_t1, t2 TO new_t2;
```

* We can also use RENAME TABLE to rename views.


* RENAME TABLE cannot be used to rename temporary tables. We can use ALTER TABLE to rename temporary tables.


* If we rename a table referenced by a view, a stored procedure, or a child table with foreign keys, then using the view, procedure, and foreign keys will be invalid.

# DROP

```mysql
DROP DATABASE [IF EXISTS] db_name;

DROP TABLE [IF EXISTS] tbl_name;

DROP TABLE tbl1, tbl2;

DROP INDEX index_name ON tbl_name;

DROP TRIGGER trigger_name;
```

# TRUNCATE

Delete all data in a table. TRUNCATE TABLE is more efficient than DELETE. 

```mysql
TRUNCATE [TABLE] tbl_name;
```

Properties:

* Truncating a table will fail, if another table has a foreign key referencing the table.


* Truncating a table cannot be rolled back.

* Truncating a table will reset value in the AUTO_INCREMENT column to its start value.

# INSERT

## INSERT INTO

```mysql
INSERT INTO tbl_name (col1, col2) VALUES (x1, x2), (y1, y2);
INSERT INTO tbl_name VALUES ROW(x1, x2), ROW(y1, y2);
INSERT INTO tbl_name SET col1 = x1, col2 = x2;
```

* If a colum is defined with a default value in creating a table, we can use the default value in INSERT:

```mysql
CREATE TABLE t (
    ...
    x INT NOT NULL DEFAULT 10
);

INSERT INTO t (x) VALUES(DEFAULT);
```

* When inserting a row to a table, the maximum size of the row is limited by the system variable `max_allowed_packet`:

```mysql
SELECT @@max_allowed_packet;     # 4194304

# or

SHOW VARIABLES LIKE 'max_allowed_packet';
```

* When a date format is wrong and sql_mode is not strict, it is still inserted to the table (with a warning), but its date value becomes 0000-00-00. In a strict sql mode, we will have an error.

```mysql
INSERT INTO t (due_date) VALUES ('2020-04-xx');      # it is inserted to the table, but its value is 0000-00-00.
```

## INSERT INTO ... SELECT ...

```mysql
INSERT INTO t1 SELECT * FROM t2 [WHERE ...];
INSERT INTO t1 (x1, y1) SELECT x2, y2 FROM t2 [WHERE ... ];
```

## INSERT INTO ... VALUES ...

```mysql
INSERT INTO t (x, y, z) VALUES(
    (SELECT MAX(*) FROM ...), 
    (SELECT yy FROM ...),
    (SELECT COUNT(*) FROM ...)
);
```

## INSERT INTO ... VALUES ... ON DUPLICATE KEY UPDATE ...

In the following, id is a column that is a UNIQUE index or PRIMARY KEY. If id_val exists in the table, the value of col of the row is replaced with col_val2. 

```mysql
INSERT INTO t (id, col) VALUES (id_val, col_val)
ON DUPLICATE KEY UPDATE col = col_val2; 
```

## INSERT IGNORE INTO ...

If an error occurs while inserting multiple rows to a table, MySQL terminates the statement, returns an error, and no rows are inserted into the table.

Using INSERT IGNORE, the invalid rows are ignored and the valid rows are inserted into the table.

```mysql
INSERT IGNORE INTO t(c) VALUES(val1), (val2);
```

In a strict sql mode, inserting an invalid row raises an error. Using INSERT IGNORE in a strict sql mode will raise a warning instead.


Assume name is a column name in the table t whose data type is VARCHAR(4). 

```mysql
INSERT INTO t VALUES('Apple');   
# error in a strict sql mode

INSERT IGNORE INTO t VALUES('Apple'); 
# warning in a stric sql mode
# its value in the table is 'Appl'.
```

# UPDATE ... SET

```mysql
UPDATE t SET x = x - y;

UPDATE t SET x1 = y1, x2 = y2 WHERE id = 10;

UPDATE t SET x = REPLACE(x, sub_str_in_x, new_str) WHERE id < 10; 

UPDATE t SET x = (SELECT ...) WHERE id > 10;
```


## UPDATE ... JOIN 

To update data from multiple tables using a single UPDATE statement, use the UPDATE ... JOIN.

In the following we join two tables t1 and t2 on t1.x = y2.y and update t1.w by t1.w = t1.w * t2.z for any row satisfying a condition:

```mysql
UPDATE t1 INNER JOIN t2 ON t2.y = t1.x
SET w = w * z
WHERE ...;
```

Using LEFT JOIN:

```mysql
UPDATE t1 LEFT JOIN t2 ON t2.y = t1.x
SET w = w * 10
WHERE t2.u IS NULL;
```

The following updates two tables. Since both tables are updated, their names should follow by UPDATE.

```mysql
UPDATE t1, t2 
INNER JOIN t2 ON t2.y = t1.x
SET t1.w = t1.w * t2.w, t2.u = 3*t1.v
WHERE ...;
```

# DELETE FROM ...

DELETE statements return the number of rows deleted. To delete all rows in a table, it's better to use `TRUNCATE TABLE`.


```mysql
DELETE FROM t;        # Delete all rows

DELETE FROM t WHERE ...;

DELETE FROM t ORDER BY c DESC LIMIT 5;

DELETE FROM t WHERE ... ORDER BY c LIMIT 3;
```

## DELETE ... JOIN ...

Use DELETE ... JOIN to delete data from multiple tables.

```mysql
DELETE t1, t2
FROM t1 INNER JOIN t2 ON t2.y = t1.x
WHERE ...

DELETE t1
FROM t1 LEFT JOIN t2 ON t2.y = t1.x
WHERE t2.y IS NULL;
```

# REPLACE [INTO] ...

If a duplicate key is found, delete the current row and insert the new one into the table. MySQL uses PRIMARY KEY or UNIQUE KEY index in using a REPLACE clause. 


In the following, if id_val already exists in the table, the row containing the id_val is removed and then the new row having (id_val, x_val) is inserted into the same position.

```mysql
REPLACE INTO t(id, x) VALUES (id_val, x_val);
```

The following raises an error:

```mysql
REPLACE INTO t(x) VALUES (x_val) WHERE id = id_val;
```

## REPLACE ... SET

The following updates a row. Here other columns not shown in the query should have default values.

```mysql
REPLACE INTO t SET id=id_val, x=x_val;
```

## REPLACE ... SELECT

```mysql
REPLACE INTO t(id, x)
SELECT id2, x2 FROM ... WHERE ...;
```

# Constraints

## PRIMARY KEY, INDEX

* MySQL implicitly adds a NOT NULL constraint to primary key columns.


* When you define a primary key for a table, MySQL automatically creates an index called PRIMARY.


* INDEX or KEY is used to create an index column or a set of index columns that is not a primary key.


* INDEX can have NULL values.


* A colmn with UNIQUE INDEX must have distinct values. 


The following shows all indexes associated with a table:

```mysql
SHOW INDEX FROM tbl_name;
```

## UNIQUE

```mysql
CREATE TABLE tbl (
    x1 data_type UNIQUE,
    x2 data_type,
    x3 data_type,
    ...
    [CONSTRAINT constraint_name]
    UNIQUE(x2,x3)
);
```

## CHECK

```mysql
CREATE TABLE tbl (
    x data_type NOT NULL CHECK(x >= 0),
    y data_type NOT NULL,
    z data_type NOT NULL,
    [CONSTRAINT constraint_name] CHECK(y >= z),
    CHECK(x*y <= z)
);
```


## FOREIGN KEY

```mysql
# Reference: dev.mysql.com

CREATE TABLE parent (
    id INT NOT NULL,
    PRIMARY KEY (id)
) ENGINE=INNODB;

CREATE TABLE child (
    id INT AUTO_INCREMENT,
    parent_id INT,
    PRIMARY KEY (id, parent_id),
    [CONSTRAINT fk_parentID)]
    FOREIGN KEY (parent_id)
        REFERENCES parent(id)
        ON DELETE CASCADE ON UPDATE RESTRICT
) ENGINE=INNODB;
```

* Inserting a row to the child table whose parent_id does not exist in the parent table will raise an error.


* A foreign key can reference the primary key in the same table (self-referencing foreign key).


* MySQL has the following reference options:
    * CASCADE: same action in the child table
    * SET NULL: set NULL in the child table
    * RESTRICT or NO ACTION (default): deleting/updating in the parent table is not allowed, if a matching row is found in a child table. 
    
    
* foreign_key_checks

It is useful to disable foreign key checks in some cases. For example, if we load data  into a table with foreign_key_checks enabled, parent tables should be loaded before child tables are loaded.

```mysql
SET foreign_key_checks = 0;     # Disable foreign key checks
SET foreign_key_checks = 1;     # Enable foreign key checks
```